In [1]:
# Import libraries
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt
import re
import string
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
from sklearn.preprocessing import StandardScaler, normalize
import math
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('Reviews.csv')
print(data.shape)
data.head(10)

(20000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,5,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,5,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,5,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,5,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [3]:
# Set of stopwords
stop = set(stopwords.words('english'))

# Initialising the snowball stemmer
sno = nltk.stem.SnowballStemmer('english') 

# Function to clean the word of any html-tags
def cleanhtml(sentence): 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext

# Function to clean the word of any punctuation or special characters
def cleanpunc(sentence): 
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

i=0
str1=' '
final_string=[]

# Store words from +ve reviews 
all_positive_words=[]

# Store words from -ve reviews
all_negative_words=[] 
s=''

final_string=[]
all_positive_words=[] 
all_negative_words=[] 
s=''
for sent in data['Text'].values:
    filtered_sentence=[]

    # Remove HTMl tags
    sent=cleanhtml(sent) 
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                else:
                    continue
            else:
                continue 
    str1 = b" ".join(filtered_sentence) #final string of cleaned words    
    final_string.append(str1)
    i+=1

In [4]:
count_vect = CountVectorizer()
final_bow_count = count_vect.fit_transform(final_string)

In [5]:
final_bow_np = StandardScaler(with_mean=False).fit_transform(final_bow_count)

In [6]:
X = final_bow_np

X_train =  final_bow_np[:math.ceil(len(data)*.9)] 
X_test = final_bow_np[math.ceil(len(data)*.9):]

In [7]:
%%time
k = 50
nbrs = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='cosine').fit(X_train)
distances, neighbors = nbrs.kneighbors(X_test, k)

CPU times: user 1.22 s, sys: 305 ms, total: 1.53 s
Wall time: 1.54 s


In [8]:
def recall(pred, gt):
    if pred.shape[0] == 1:
        return len(set(pred).intersection(set(gt))/len(gt))
    assert pred.shape[0] == gt.shape[0]
    return np.array([len(set(pred[i]).intersection(set(gt[i])))/len(gt[i]) for i in range(len(pred))]).mean()

Using hnswLib for NN Search

In [16]:
import hnswlib
import datetime

X_tr = normalize(X_train.toarray())
X_tst = normalize(X_test.toarray())

start = datetime.datetime.now()
idx = hnswlib.Index(space = 'l2', dim = X_tr.shape[1])
idx.init_index(max_elements=X_tr.shape[0],ef_construction=400,M=50)
idx.add_items(X_tr)

nbrs,dists = idx.knn_query(X_tst,k=50)
end = datetime.datetime.now()

print("hnsw recall val: ",recall(nbrs,neighbors))
print("Duration: {}".format(end - start))

hnsw recall val:  0.84999
Duration: 0:05:55.119373


Using Annoy Package for NN Search

In [18]:
from annoy import AnnoyIndex
import datetime

start = datetime.datetime.now()
idx = AnnoyIndex(X_tr.shape[1],metric="angular")
for i,x in enumerate(X_tr):
    idx.add_item(i,x)
idx.build(50)

nbrs = []
dists = []

for v in X_tst:
    nid,dist = idx.get_nns_by_vector(v,50,include_distances=True)
    nbrs.append(nid)
    dists.append(dist)
end = datetime.datetime.now()

print("hnsw recall val: ",recall(np.array(nbrs),neighbors))
print("Duration: {}".format(end - start))